[Link a la carpeta con todo el TP](https://drive.google.com/drive/u/0/folders/1do-iyf2SzQln-fh8tmu9mSfzLuim5xs5)

# Imports & Download stage

In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 861.5 kB/s eta 0:00:00


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from category_encoders import BinaryEncoder
import pandas as pd
import numpy as np
%matplotlib inline
import pyarrow.parquet as pq
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from scipy.stats import uniform, randint
from sklearn.metrics import f1_score

pd.options.display.float_format = '{:20,.10f}'.format

import gc

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

ACLARACIÓN: Como usé el mismo feature engineering para ambos modelos, lo hice en [otro colab](https://colab.research.google.com/drive/1oolZwdMmiiPSBROS8q7c9taycd5NoJgR#scrollTo=XeHTXwYzXc3o) y me guardé ese dataset (que es el que estoy cargando aquí)

In [5]:
if not os.path.exists('test.parq'):
  id1="18FlKfUK0ItadAP1jz9KG96KrbWyiQ1zo"
  downloaded1 = drive.CreateFile({'id': id1})
  downloaded1.GetContentFile('test.parq')

if not os.path.exists('train.parq'):
  id2="1H67jcnBBH5eA6elQu95fEDQwWzOuK5Gr"
  downloaded2 = drive.CreateFile({'id': id2})
  downloaded2.GetContentFile('train.parq')

# Init stage

load dataframe

In [6]:
raw_df = pq.read_table('train.parq').to_pandas()
test_df = pq.read_table('test.parq').to_pandas()

Train/test split based on IPs

In [7]:
# Create a DataFrame with unique IPs and their corresponding labels
ip_label_df = raw_df[['attacker_ip_enum', 'label']].drop_duplicates()

# Split the unique IPs into two sets with roughly 80%-20% proportions, stratified by label
train_ips, test_ips = train_test_split(ip_label_df['attacker_ip_enum'],
                                       test_size=0.2,
                                       random_state=42,
                                       stratify=ip_label_df['label'])

# Use these sets to filter the original DataFrame
train_df = raw_df[raw_df['attacker_ip_enum'].isin(train_ips)]
val_df = raw_df[raw_df['attacker_ip_enum'].isin(test_ips)]

del raw_df
del ip_label_df
gc.collect()

64

# Encoding stage

Para el encoding traté de mantener la mayor cantidad de información posible de los dataframes. Por eso sólo usé mean cuando había demasiadas categorías.

Además, las categorías mean-encodeadas en el preceptron eran muy importantes, por lo que me pareció que beneficiaría al modelo tenerlas así

Por como me quedó armado el dataset, no tuve que tener ninguna consideración especial por usar un modelo de árboles ya que las únicas columnas que no son un mean/OHE/binary son las del día que están encodeadas con sinoidales

### Drop IP to avoid overfitting/target leak

In [8]:
train_df.drop(['attacker_ip_enum'], axis=1, inplace=True)
val_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

test_ips = test_df['attacker_ip_enum'] # I'll need these for kaggle submission
test_df.drop(['attacker_ip_enum'], axis=1, inplace=True)

### Countries (binary)

In [9]:
# Initialize a BinaryEncoder for the 'watcher_country' column
binary_encoder = BinaryEncoder(cols=['watcher_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['watcher_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['watcher_country']])
val_df_encoded = binary_encoder.transform(val_df[['watcher_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'watcher_country' column
train_df.drop('watcher_country', axis=1, inplace=True)
test_df.drop('watcher_country', axis=1, inplace=True)
val_df.drop('watcher_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['watcher_country_0'] = train_df['watcher_country_0'].astype('uint8')
test_df['watcher_country_0'] = test_df['watcher_country_0'].astype('uint8')
val_df['watcher_country_0'] = val_df['watcher_country_0'].astype('uint8')

train_df['watcher_country_1'] = train_df['watcher_country_1'].astype('uint8')
test_df['watcher_country_1'] = test_df['watcher_country_1'].astype('uint8')
val_df['watcher_country_1'] = val_df['watcher_country_1'].astype('uint8')

train_df['watcher_country_2'] = train_df['watcher_country_2'].astype('uint8')
test_df['watcher_country_2'] = test_df['watcher_country_2'].astype('uint8')
val_df['watcher_country_2'] = val_df['watcher_country_2'].astype('uint8')

train_df['watcher_country_3'] = train_df['watcher_country_3'].astype('uint8')
test_df['watcher_country_3'] = test_df['watcher_country_3'].astype('uint8')
val_df['watcher_country_3'] = val_df['watcher_country_3'].astype('uint8')

train_df['watcher_country_4'] = train_df['watcher_country_4'].astype('uint8')
test_df['watcher_country_4'] = test_df['watcher_country_4'].astype('uint8')
val_df['watcher_country_4'] = val_df['watcher_country_4'].astype('uint8')

In [10]:
# Initialize a BinaryEncoder for the 'attacker_country' column
binary_encoder = BinaryEncoder(cols=['attacker_country'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attacker_country']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attacker_country']])
val_df_encoded = binary_encoder.transform(val_df[['attacker_country']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attacker_country' column
train_df.drop('attacker_country', axis=1, inplace=True)
test_df.drop('attacker_country', axis=1, inplace=True)
val_df.drop('attacker_country', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attacker_country_0'] = train_df['attacker_country_0'].astype('uint8')
test_df['attacker_country_0'] = test_df['attacker_country_0'].astype('uint8')
val_df['attacker_country_0'] = val_df['attacker_country_0'].astype('uint8')

train_df['attacker_country_1'] = train_df['attacker_country_1'].astype('uint8')
test_df['attacker_country_1'] = test_df['attacker_country_1'].astype('uint8')
val_df['attacker_country_1'] = val_df['attacker_country_1'].astype('uint8')

train_df['attacker_country_2'] = train_df['attacker_country_2'].astype('uint8')
test_df['attacker_country_2'] = test_df['attacker_country_2'].astype('uint8')
val_df['attacker_country_2'] = val_df['attacker_country_2'].astype('uint8')

train_df['attacker_country_3'] = train_df['attacker_country_3'].astype('uint8')
test_df['attacker_country_3'] = test_df['attacker_country_3'].astype('uint8')
val_df['attacker_country_3'] = val_df['attacker_country_3'].astype('uint8')

train_df['attacker_country_4'] = train_df['attacker_country_4'].astype('uint8')
test_df['attacker_country_4'] = test_df['attacker_country_4'].astype('uint8')
val_df['attacker_country_4'] = val_df['attacker_country_4'].astype('uint8')

### AS Names (Mean/Target)

In [11]:
target_encoder = TargetEncoder(cols=['watcher_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_as_name']])
val_df_encoded = target_encoder.transform(val_df[['watcher_as_name']])

train_df[['watcher_as_name']] = train_df_encoded.astype('float32')
test_df[['watcher_as_name']] = test_df_encoded.astype('float32')
val_df[['watcher_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

In [12]:
target_encoder = TargetEncoder(cols=['attacker_as_name'])

train_df_encoded = target_encoder.fit_transform(train_df[['attacker_as_name']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['attacker_as_name']])
val_df_encoded = target_encoder.transform(val_df[['attacker_as_name']])

train_df[['attacker_as_name']] = train_df_encoded.astype('float32')
test_df[['attacker_as_name']] = test_df_encoded.astype('float32')
val_df[['attacker_as_name']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Watcher UUID Enums (Mean/Target)

In [13]:
target_encoder = TargetEncoder(cols=['watcher_uuid_enum'])

train_df_encoded = target_encoder.fit_transform(train_df[['watcher_uuid_enum']], train_df['label'])
test_df_encoded = target_encoder.transform(test_df[['watcher_uuid_enum']])
val_df_encoded = target_encoder.transform(val_df[['watcher_uuid_enum']])

train_df[['watcher_uuid_enum']] = train_df_encoded.astype('float32')
test_df[['watcher_uuid_enum']] = test_df_encoded.astype('float32')
val_df[['watcher_uuid_enum']] = val_df_encoded.astype('float32')

del train_df_encoded, test_df_encoded, val_df_encoded, target_encoder
gc.collect()

0

### Attack type (OHE/Bin)

In [14]:
#Not sure if I want to use this yet

train_df.drop(['attack_type'], axis=1, inplace=True)
test_df.drop(['attack_type'], axis=1, inplace=True)
val_df.drop(['attack_type'], axis=1, inplace=True)

In [15]:
# Initialize a BinaryEncoder for the 'attack_service' column
binary_encoder = BinaryEncoder(cols=['attack_service'])

# Fit and transform on the training data
train_df_encoded = binary_encoder.fit_transform(train_df[['attack_service']])

# Transform the test data
test_df_encoded = binary_encoder.transform(test_df[['attack_service']])
val_df_encoded = binary_encoder.transform(val_df[['attack_service']])

# Merge the encoded columns back into the original DataFrames
train_df = pd.concat([train_df, train_df_encoded], axis=1)
test_df = pd.concat([test_df, test_df_encoded], axis=1)
val_df = pd.concat([val_df, val_df_encoded], axis=1)

# Clean up unnecessary variables
del train_df_encoded, test_df_encoded, val_df_encoded
gc.collect()

# Drop the original 'attack_service' column
train_df.drop('attack_service', axis=1, inplace=True)
test_df.drop('attack_service', axis=1, inplace=True)
val_df.drop('attack_service', axis=1, inplace=True)

# Cast to uint8 for efficiency
train_df['attack_service_0'] = train_df['attack_service_0'].astype('uint8')
test_df['attack_service_0'] = test_df['attack_service_0'].astype('uint8')
val_df['attack_service_0'] = val_df['attack_service_0'].astype('uint8')

train_df['attack_service_1'] = train_df['attack_service_1'].astype('uint8')
test_df['attack_service_1'] = test_df['attack_service_1'].astype('uint8')
val_df['attack_service_1'] = val_df['attack_service_1'].astype('uint8')

train_df['attack_service_2'] = train_df['attack_service_2'].astype('uint8')
test_df['attack_service_2'] = test_df['attack_service_2'].astype('uint8')
val_df['attack_service_2'] = val_df['attack_service_2'].astype('uint8')

train_df['attack_service_3'] = train_df['attack_service_3'].astype('uint8')
test_df['attack_service_3'] = test_df['attack_service_3'].astype('uint8')
val_df['attack_service_3'] = val_df['attack_service_3'].astype('uint8')

In [16]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_category' column in both train and test DataFrames
encoded_attack_category_train = encoder.fit_transform(train_df[['attack_category']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_category_train = encoded_attack_category_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_category' column
encoded_attack_category_train_df = pd.DataFrame(encoded_attack_category_train, columns=encoder.get_feature_names_out(['attack_category']))

# Concatenate the one-hot encoded 'attack_category' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_train_df)

# Run GC to save memory
del encoded_attack_category_train, encoded_attack_category_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_category_test = encoder.transform(test_df[['attack_category']])
encoded_attack_category_test = encoded_attack_category_test.todense().astype('int8')
encoded_attack_category_test_df = pd.DataFrame(encoded_attack_category_test, columns=encoder.get_feature_names_out(['attack_category']))
test_df = test_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_test_df)
del encoded_attack_category_test, encoded_attack_category_test_df
gc.collect()

encoded_attack_category_val = encoder.transform(val_df[['attack_category']])
encoded_attack_category_val = encoded_attack_category_val.todense().astype('int8')
encoded_attack_category_val_df = pd.DataFrame(encoded_attack_category_val, columns=encoder.get_feature_names_out(['attack_category']))
val_df = val_df.reset_index().drop(['index', 'attack_category'],axis=1).join(encoded_attack_category_val_df)
del encoded_attack_category_val, encoded_attack_category_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Attack time of day (OHE)

In [17]:
# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse=True, drop='first')  # Use sparse=True

# Fit and transform the 'attack_time_of_day' column in both train and test DataFrames
encoded_attack_time_of_day_train = encoder.fit_transform(train_df[['attack_time_of_day']]
                                                      )
# Convert the sparse matrices to dense arrays of integers
encoded_attack_time_of_day_train = encoded_attack_time_of_day_train.todense().astype('int8')

# Create DataFrames for the one-hot encoded 'attack_time_of_day' column
encoded_attack_time_of_day_train_df = pd.DataFrame(encoded_attack_time_of_day_train, columns=encoder.get_feature_names_out(['attack_time_of_day']))

# Concatenate the one-hot encoded 'attack_time_of_day' columns to the respective DataFrames
train_df = train_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_train_df)

# Run GC to save memory
del encoded_attack_time_of_day_train, encoded_attack_time_of_day_train_df
gc.collect()

# Do it again for other 2 dataframes

encoded_attack_time_of_day_test = encoder.transform(test_df[['attack_time_of_day']])
encoded_attack_time_of_day_test = encoded_attack_time_of_day_test.todense().astype('int8')
encoded_attack_time_of_day_test_df = pd.DataFrame(encoded_attack_time_of_day_test, columns=encoder.get_feature_names_out(['attack_time_of_day']))
test_df = test_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_test_df)
del encoded_attack_time_of_day_test, encoded_attack_time_of_day_test_df
gc.collect()

encoded_attack_time_of_day_val = encoder.transform(val_df[['attack_time_of_day']])
encoded_attack_time_of_day_val = encoded_attack_time_of_day_val.todense().astype('int8')
encoded_attack_time_of_day_val_df = pd.DataFrame(encoded_attack_time_of_day_val, columns=encoder.get_feature_names_out(['attack_time_of_day']))
val_df = val_df.reset_index().drop(['index', 'attack_time_of_day'],axis=1).join(encoded_attack_time_of_day_val_df)
del encoded_attack_time_of_day_val, encoded_attack_time_of_day_val_df
gc.collect()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

### Checkpoint!

In [18]:
train_df.sample(5)

,watcher_as_name,attacker_as_name,watcher_uuid_enum,label,attack_day_sin,attack_day_cos,same_country,watcher_country_0,watcher_country_1,watcher_country_2,...,attack_service_1,attack_service_2,attack_service_3,attack_category_crawl,attack_category_exploit,attack_category_scan,attack_category_spam,attack_category_unknown,attack_time_of_day_day,attack_time_of_day_night
4356436,0.0220104903,0.0000288600,0.0215269011,0,-0.7818315029,0.6234897971,0,1,0,1,...,0,0,1,0,0,1,0,0,0,0
8397237,0.0178826097,0.0003418816,0.0131789800,0,0.9749279022,-0.2225209326,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
26702649,0.0003070782,0.0000000000,0.0000000000,0,0.9749279022,-0.2225209326,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
26694494,0.0036787246,0.0000000000,0.0000000000,0,-0.0000000000,1.0000000000,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
28829652,0.0156143345,0.0359005593,0.0225080382,0,-0.9749279022,-0.2225209326,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1


# Model Stage

In [19]:
# I need all memory I can get
test_df.to_parquet('test_processed.parq')
del test_df
gc.collect()

0

### Hiperparam search

Metodología de la búsqueda: me guardé el score para los defaults (lo único que cambié para la 'base' fue el `n_estimators`=10), y luego puse rangos que me parecieron aceptables alrededor de esos parámetros.

Luego lo dejé corriendo y que me devolviera todos las combinaciones que superaran esa base, y repetir eso para varias combinaciones de los parámetros más 'categóricos'.

El problema de hacer este tipo de cosas con random forest es que tarda *mucho más* que SGD en entrenar (~15 minutos por fit vs. ~30 segundos), por lo que no podía probar muchas combinaciones. Este problema se ve claro en los rangos de `n_estimators`: números más altos podrían darme mejores resultados, pero el tiempo que tardaría en fitear crecería de más (con el default de 100 tarda más de una hora)

### Prueba con un slice de 0's y 3 slices de 1's

In [24]:
# Define hyperparameters
n_batches = 6 # ~5M batch size
n_max_iter = 1000
cut_score = 0.40

# Split the data into features (X) and target variable (y)
X1_train = train_df[train_df['label'] == 1].drop('label', axis=1)
X0_train = train_df[train_df['label'] == 0].drop('label', axis=1).sample(int(X1_train.shape[0]*2), random_state=42)
X_train = pd.concat([X1_train, X0_train], axis=0)
X_train = X_train.sample(frac=1, random_state=42).reset_index(drop=True)

del X0_train, X1_train
gc.collect()

y_train = train_df['label'].sample(X_train.shape[0], random_state=42)

X_test = val_df.drop('label', axis=1)
y_test = val_df['label']

crit_opt = ['gini', 'entropy', 'log_loss']
mf_opt = ['sqrt', 'log2']

# Instantiate the XGBoost model
for n_iter in range(n_max_iter):

  params = {
  'n_estimators': random.randint(15, 25),
  'criterion': crit_opt[random.randint(0, len(crit_opt)-1)],
  'max_depth': random.randint(5, 15),
  'min_samples_split': random.randint(2, 8),
  'min_samples_leaf': random.randint(1, 5),
  'min_weight_fraction_leaf': 0,
  'max_features': mf_opt[random.randint(0, len(mf_opt)-1)]
  }

  rf_model = RandomForestClassifier(
      n_estimators=params['n_estimators'],
      criterion=params['criterion'],
      max_depth = params['max_depth'],
      min_samples_split = params['min_samples_split'],
      min_samples_leaf = params['min_samples_leaf'],
      min_weight_fraction_leaf = params['min_weight_fraction_leaf'],
      max_features = params['max_features'],
      warm_start=True,
      random_state = 42
  )

  rf_model.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = rf_model.predict(X_test)

  # Evaluate the model
  f1_accuracy = f1_score(y_test, y_pred)

  if f1_accuracy > -1:
    max_score = f1_accuracy
    print(f'F1 Accuracy: {f1_accuracy}')
    print(f'Best Params: {params}')

F1 Accuracy: 0.0
Best Params: {'n_estimators': 21, 'criterion': 'entropy', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 19, 'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 5, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 17, 'criterion': 'log_loss', 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 23, 'criterion': 'log_loss', 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 18, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}
F1 Accuracy: 0.0
Best Params: {

KeyboardInterrupt: ignored

### Por cada slice de 0's agarrar 3 slices random de 1's

In [ ]:
# Define hyperparameters
n_batches = 6 # ~5M batch size
n_max_iter = 1000
cut_score = 0.40

# Split the data into features (X) and target variable (y)
X0_train = train_df[train_df['label'] == 0].drop('label', axis=1)
X1_train = train_df[train_df['label'] == 1].drop('label', axis=1)
y_train = train_df['label']

X0_batch_size = int(np.floor(X0_train.shape[0]/n_batches))
X1_batch_size = int(np.floor(X1_train.shape[0]/n_batches))
y_batch_size = X0_batch_size + X1_batch_size*3

X_test = val_df.drop('label', axis=1)
y_test = val_df['label']

crit_opt = ['gini', 'entropy', 'log_loss']
mf_opt = ['sqrt', 'log2']

# Instantiate the XGBoost model
for n_iter in range(n_max_iter):

  params = {
  'n_estimators': random.randint(30, 50),
  'criterion': crit_opt[random.randint(0, len(crit_opt)-1)],
  'max_depth': random.randint(100, 1000),
  'min_samples_split': random.randint(2, 8),
  'min_samples_leaf': random.randint(1, 5),
  'min_weight_fraction_leaf': 0,
  'max_features': mf_opt[random.randint(0, len(mf_opt)-1)]
  }

  d_estimators = int(params['n_estimators']/(n_batches))
  rf_model = RandomForestClassifier(
      n_estimators = d_estimators,
      criterion=params['criterion'],
      max_depth = params['max_depth'],
      min_samples_split = params['min_samples_split'],
      min_samples_leaf = params['min_samples_leaf'],
      min_weight_fraction_leaf = params['min_weight_fraction_leaf'],
      max_features = params['max_features'],
      warm_start=True,
      random_state = 42
  )

  # Train the model in batches, keeping label proportion
  for i in range(0, n_batches):
    X0_batch = X0_train.iloc[X0_batch_size*i: X0_batch_size*i + X0_batch_size]

    X1_slices = random.sample(range(0, n_batches), 3)

    X1_batch0 = X1_train.iloc[X1_batch_size*X1_slices[0]: X1_batch_size*X1_slices[0] + X1_batch_size]
    X1_batch1 = X1_train.iloc[X1_batch_size*X1_slices[1]: X1_batch_size*X1_slices[1] + X1_batch_size]
    X1_batch2 = X1_train.iloc[X1_batch_size*X1_slices[2]: X1_batch_size*X1_slices[2] + X1_batch_size]
    X_batch = pd.concat([X1_batch0, X1_batch1, X1_batch2, X0_batch], axis=0)

    del X0_batch, X1_batch0, X1_batch1, X1_batch2
    gc.collect()

    # Shuffle. Por si acaso probé con y sin esta linea
    X_batch = X_batch.sample(frac=1, random_state=42).reset_index(drop=True)

    y_batch = y_train.iloc[0:y_batch_size]

    rf_model.n_estimators = d_estimators*(i+1)
    rf_model.fit(X_batch, y_batch)

  # Make predictions on the test set
  y_pred = rf_model.predict(X_test)

  # Evaluate the model
  f1_accuracy = f1_score(y_test, y_pred)

  if f1_accuracy > -1:
    max_score = f1_accuracy
    print(f'F1 Accuracy: {f1_accuracy}')
    print(f'Best Params: {params}')

F1 Accuracy: 0.0
Best Params: {'n_estimators': 39, 'criterion': 'entropy', 'max_depth': 917, 'min_samples_split': 5, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 32, 'criterion': 'gini', 'max_depth': 730, 'min_samples_split': 3, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 46, 'criterion': 'gini', 'max_depth': 492, 'min_samples_split': 6, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}
F1 Accuracy: 0.0
Best Params: {'n_estimators': 31, 'criterion': 'entropy', 'max_depth': 470, 'min_samples_split': 6, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}
F1 Accuracy: 1.225332524613867e-05
Best Params: {'n_estimators': 41, 'criterion': 'log_loss', 'max_depth': 432, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}
F1 Accuracy:

### Viejo encoding

F1 Accuracy: 0.40173480708873976

Best Params: {'n_estimators': 10, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4152071467807684

Best Params: {'n_estimators': 12, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4282742288437903

Best Params: {'n_estimators': 12, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4330884113216581

Best Params: {'n_estimators': 15, 'max_depth': None, 'min_samples_split': 8, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.43428837699801964

Best Params: {'n_estimators': 16, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.436724687123616

Best Params: {'n_estimators': 17, 'max_depth': None, 'min_samples_split': 7, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

### Nuevo Encoding

F1 Accuracy: 0.4825603285983227

Best Params: {'n_estimators': 28, 'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt', 'bootstrap': True}

---

F1 Accuracy: 0.4726692482827064

Best Params: {'n_estimators': 32, 'criterion': 'gini', 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt', 'bootstrap': True}

---

F1 Accuracy: 0.4749464390053886

Best Params: {'n_estimators': 29, 'max_depth': None, 'min_samples_split': 7, 'min_samples_leaf': 4, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4715637291378761

Best Params: {'n_estimators': 22, 'max_depth': None, 'min_samples_split': 8, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'log2'}

---

F1 Accuracy: 0.4665967936027328

Best Params: {'n_estimators': 25, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4609627097932141

Best Params: {'n_estimators': 17, 'max_depth': None, 'min_samples_split': 7, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

### Reentrega

F1 Accuracy: 0.44605808486469245

Best Params: {'n_estimators': 31, 'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 8, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}

---

F1 Accuracy: 0.4333142025975288

Best Params: {'n_estimators': 22, 'criterion': 'log_loss', 'max_depth': 87, 'min_samples_split': 6, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt'}




### Kaggle submission

In [ ]:
# Train model w/ best params

batch_size = 6000000

# Split the data into features (X) and target variable (y)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = val_df.drop('label', axis=1)
y_test = val_df['label']


params =  {
    'n_estimators': 28,
    'criterion': 'gini',
    'max_depth': 13,
    'min_samples_split': 7,
    'min_samples_leaf': 2,
    'min_weight_fraction_leaf': 0,
    'max_features': 'sqrt',
    'bootstrap': True
}

rf_model = RandomForestClassifier(
    n_estimators = params['n_estimators'],
    criterion=params['criterion'],
    max_depth = params['max_depth'],
    min_samples_split = params['min_samples_split'],
    min_samples_leaf = params['min_samples_leaf'],
    min_weight_fraction_leaf = params['min_weight_fraction_leaf'],
    max_features = params['max_features'],
    bootstrap=params['bootstrap'],
    warm_start=True,
    random_state = 42
)

# Train the model in batches
for i in range(0, len(X_train), batch_size):
    X_batch = X_train.iloc[i:i+batch_size]
    y_batch = y_train.iloc[i:i+batch_size]

    rf_model.fit(X_batch, y_batch)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
f1_accuracy = f1_score(y_test, y_pred)

print(f'F1 Accuracy: {f1_accuracy}')
print(f'Best Params: {params}')


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:452: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:452: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:452: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:452: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:452: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(


F1 Accuracy: 0.4536634496474878
Best Params: {'n_estimators': 28, 'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0, 'max_features': 'sqrt', 'bootstrap': True}


In [ ]:
test_df = pq.read_table('test_processed.parq').to_pandas()

In [21]:
# in case I need them again
train_df = pq.read_table('train_encoded.parq').to_pandas()
val_df = pq.read_table('val_encoded.parq').to_pandas()

In [20]:
train_df.to_parquet('train_encoded.parq')
val_df.to_parquet('val_encoded.parq')
del train_df, val_df
gc.collect()

0

In [ ]:
kaggle_pred = rf_model.predict(test_df)
kaggle_df = pd.DataFrame({'attacker_ip_enum': test_ips, 'prediction': kaggle_pred})

In [ ]:
kaggle_df = kaggle_df.groupby('attacker_ip_enum').mean().reset_index()
kaggle_df['prediction'] = kaggle_df['prediction'].round().astype(int)

In [ ]:
kaggle_df.set_index('attacker_ip_enum').to_csv('rf_submission.csv')